In [7]:
import cv2
import os
import shutil
import numpy as np
import pandas as pd
import mediapipe as mp
from PIL import Image
import imageio
import natsort
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [13]:
order_number = '01'
   
npy_video_save_path = r'C:\Users\user\Documents\0_2024_signLang_simul\npy_video' \
                + '\\' + order_number + '_real_word_npy_all_frame_video'
npy_img_save_path = r'C:\Users\user\Documents\0_2024_signLang_simul\npy_img' \
                + '\\' + order_number + '_real_word_npy_all_frame_img'

os.makedirs(npy_video_save_path, exist_ok=True)
os.makedirs(npy_img_save_path, exist_ok=True)

error_path = r'C:\Users\user\Documents\0_2024_signLang_simul\npy_error'

# error_txt = open(error_path + '\\' + npy_video_save_path.split('\\')[-1] + '.txt', 'a')
# error_txt = open(error_path + '\\' + npy_img_save_path.split('\\')[-1] + '.txt', 'a')

video_path = r'C:\Users\user\Documents\0_signlang_download\수어 영상\1.Training' \
            + '\\' + order_number
            
img_tmp_path = r'C:\Users\user\Documents\0_signlang_download\img_tmp'

Label = pd.read_csv(r'C:\Users\user\Documents\0_2024_signLang_simul\labels' \
                    + '\\' + order_number + '_real_word_label.csv')

video_list = os.listdir(video_path)
print('## video folder first name', video_list[0], '\n')

one_word = '가깝다'
video_d = ['_F.np4', '_U.mp4', '_D.mp4']

new_video_list = Label[Label['label'] == one_word]
new_video_list = new_video_list[new_video_list['name'].str.contains('_F.mp4') | 
                                new_video_list['name'].str.contains('_U.mp4') | \
                                new_video_list['name'].str.contains('_D.mp4')]

print('## origin video list len : ', len(video_list))
print('## new_video_list_len : ', len(new_video_list))
print('## save video npy folder : ', npy_video_save_path.split('\\')[-1])
print('## save img npy folder : ', npy_img_save_path.split('\\')[-1])
print('\n## new label data')
print(new_video_list.head())

## video folder first name NIA_SL_WORD0001_REAL16_D.mp4 

## origin video list len :  15000
## new_video_list_len :  3
## save video npy folder :  16_real_word_npy_all_frame_video
## save img npy folder :  16_real_word_npy_all_frame_img

## new label data
                              name  duration  start    end label
4735  NIA_SL_WORD0948_REAL16_D.mp4     3.834  1.359  2.554   가깝다
4736  NIA_SL_WORD0948_REAL16_F.mp4     3.834  1.359  2.554   가깝다
4739  NIA_SL_WORD0948_REAL16_U.mp4     3.834  1.359  2.554   가깝다


In [14]:
error_count = 0

for k, j in enumerate(tqdm(new_video_list['name'])):
    # if k == 1:
    #     break
    # cap = cv2.VideoCapture('./sample/videos/가깝다.mp4')
    cap = cv2.VideoCapture(os.path.join(video_path, j))

    right_keypoints = []
    left_keypoints = []

    frame_count = 0

    duration = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    # print('## duration is ', duration)

    both_hands = np.zeros((int(duration), 42, 3))

    with mp_hands.Hands(
        model_complexity = 0,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                break
            
            cap_img_save_path = os.path.join(img_tmp_path, str(frame_count) + '.png')
            cv2.imwrite(cap_img_save_path, image)
            
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = hands.process(image)
            
            right_keypoint = np.zeros((21, 3))
            left_keypoint = np.zeros((21, 3))

            if results.multi_hand_landmarks is not None:
                if len(results.multi_hand_landmarks) == 2:
                    for i in range(21):
                        if results.multi_hand_landmarks[0].landmark[0].x > results.multi_hand_landmarks[1].landmark[0].x:
                            right_keypoint[i, 0] = results.multi_hand_landmarks[0].landmark[i].x
                            right_keypoint[i, 1] = results.multi_hand_landmarks[0].landmark[i].y
                            right_keypoint[i, 2] = results.multi_hand_landmarks[0].landmark[i].z

                            left_keypoint[i, 0] = results.multi_hand_landmarks[1].landmark[i].x
                            left_keypoint[i, 1] = results.multi_hand_landmarks[1].landmark[i].y
                            left_keypoint[i, 2] = results.multi_hand_landmarks[1].landmark[i].z
                        else:
                            right_keypoint[i, 0] = results.multi_hand_landmarks[1].landmark[i].x
                            right_keypoint[i, 1] = results.multi_hand_landmarks[1].landmark[i].y
                            right_keypoint[i, 2] = results.multi_hand_landmarks[1].landmark[i].z

                            left_keypoint[i, 0] = results.multi_hand_landmarks[0].landmark[i].x
                            left_keypoint[i, 1] = results.multi_hand_landmarks[0].landmark[i].y
                            left_keypoint[i, 2] = results.multi_hand_landmarks[0].landmark[i].z
            
            left_keypoint = np.array(left_keypoint)
            right_keypoint = np.array(right_keypoint)
            both_hand = np.concatenate((left_keypoint, right_keypoint), axis=0)
            # print(both_hand.shape)
            # print(both_hand)
            both_hands[frame_count] = both_hand
            
            frame_count += 1
        
        try:
            save_file = np.array(both_hands)
            # print('## both_hands shape is', both_hands.shape)
            save_file = save_file.flatten()
            np.save(npy_video_save_path + f'\\{j[:-4]}.npy', save_file)
        except:
            print(j)
            error_count += 1
            
            
    ## image to mediapipe
    png_lst = os.listdir(img_tmp_path)

    IMAGE_FILES = [img_tmp_path + '//' + x for x in png_lst]
    IMAGE_FILES = natsort.natsorted(png_lst)

    # print(IMAGE_FILES[0])

    right_keypoints = []
    left_keypoints = []
    both_hands = np.zeros((len(png_lst), 42, 3))

    frame_count = 0

    with mp_hands.Hands(
        static_image_mode=True,
        max_num_hands=2,
        min_detection_confidence=0.5) as hands:
        for idx, file in enumerate(IMAGE_FILES):
            # a_image = cv2.flip(cv2.imread(file), 1)
            a_image = cv2.imread(img_tmp_path + '/' + file)
            
            image = a_image.copy()
            
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = hands.process(image)
            
            if not results.multi_hand_landmarks:
                continue
            
            right_keypoint = np.zeros((21, 3))
            left_keypoint = np.zeros((21, 3))

            if results.multi_hand_landmarks is not None:
                if len(results.multi_hand_landmarks) == 2:
                    for i in range(21):
                        if results.multi_hand_landmarks[0].landmark[0].x > results.multi_hand_landmarks[1].landmark[0].x:
                            right_keypoint[i, 0] = results.multi_hand_landmarks[0].landmark[i].x
                            right_keypoint[i, 1] = results.multi_hand_landmarks[0].landmark[i].y
                            right_keypoint[i, 2] = results.multi_hand_landmarks[0].landmark[i].z

                            left_keypoint[i, 0] = results.multi_hand_landmarks[1].landmark[i].x
                            left_keypoint[i, 1] = results.multi_hand_landmarks[1].landmark[i].y
                            left_keypoint[i, 2] = results.multi_hand_landmarks[1].landmark[i].z
                        else:
                            right_keypoint[i, 0] = results.multi_hand_landmarks[1].landmark[i].x
                            right_keypoint[i, 1] = results.multi_hand_landmarks[1].landmark[i].y
                            right_keypoint[i, 2] = results.multi_hand_landmarks[1].landmark[i].z

                            left_keypoint[i, 0] = results.multi_hand_landmarks[0].landmark[i].x
                            left_keypoint[i, 1] = results.multi_hand_landmarks[0].landmark[i].y
                            left_keypoint[i, 2] = results.multi_hand_landmarks[0].landmark[i].z
            
            left_keypoint = np.array(left_keypoint)
            right_keypoint = np.array(right_keypoint)
            both_hand = np.concatenate((left_keypoint, right_keypoint), axis=0)
            # print(both_hand.shape)
            # print(both_hand)
            both_hands[idx] = both_hand
            
            frame_count += 1
        
        try:
            save_file = np.array(both_hands)
            # print('## both_hands shape is', both_hands.shape)
            save_file = save_file.flatten()
            np.save(npy_img_save_path + f'\\{j[:-4]}.npy', save_file)
        except:
            print(j)
            error_count += 1
    shutil.rmtree(img_tmp_path)
    os.makedirs(img_tmp_path, exist_ok=True)
        
    
cap.release()
print()
print('## Process No.', order_number)
print('## Error Count : ', error_count)
print('## Done')

100%|██████████| 3/3 [00:52<00:00, 17.44s/it]


## Process No. 16
## Error Count :  0
## Done
